In [24]:
import pathlib
import cadquery as cq
from jupyter_cadquery import show

from optimumSheetParser import OptimumSheetParser
from step_cadder import carAssembly




In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from ipyfilechooser import FileChooser

# --- Widget 1: Create Project ---
create_folder_chooser = FileChooser(os.getcwd())
create_folder_chooser.title = '<b>Select parent directory for new project</b>'
create_folder_chooser.show_only_dirs = True
create_folder_chooser.use_dir_icons = True
create_folder_chooser.show_hidden = False

create_project_name = widgets.Text(
    value='',
    placeholder='Enter new project name',
    description='Project Name:',
    layout=widgets.Layout(width='50%')
)

create_project_status = widgets.Output()

folder_created_callbacks = []

def on_create_clicked(b=None):
    with create_project_status:
        clear_output()
        parent = create_folder_chooser.selected_path
        name = create_project_name.value.strip()
        if not parent:
            print('Please select a parent directory.')
            return
        if not name:
            print('Please enter a project name.')
            return
        folder = os.path.join(parent, name)
        try:
            os.makedirs(os.path.join(folder, '3D'), exist_ok=True)
            os.makedirs(os.path.join(folder, 'results'), exist_ok=True)
            os.makedirs(os.path.join(folder, 'excel'), exist_ok=True)
            print(f'Created or loaded project folder: {folder}')
            print(f'  ├── 3D/')
            print(f'  └── results/')
            print(f'  └── excel/')

            # Trigger any registered callbacks
            for cb in folder_created_callbacks:
                cb(folder)
        except Exception as e:
            print(f'Error creating folder: {e}')

create_button = widgets.Button(description='Create New Project', button_style='success')
create_button.on_click(on_create_clicked)

create_project_ui = widgets.VBox([
    create_folder_chooser,
    create_project_name,
    create_button,
    create_project_status
])

# --- Widget 2: Load Project ---
load_folder_chooser = FileChooser(os.getcwd())
load_folder_chooser.title = '<b>Select existing project folder to load</b>'
load_folder_chooser.show_only_dirs = True
load_folder_chooser.use_dir_icons = True
load_folder_chooser.show_hidden = False

load_project_status = widgets.Output()

def on_load_clicked(b=None):
    with load_project_status:
        clear_output()
        folder = load_folder_chooser.selected_path
        if not folder or not os.path.isdir(folder):
            print('Folder does not exist. Please select a valid project folder.')
            return
        print(f'Loaded project folder: {folder}')
        print(f'  ├── 3D/')
        print(f'  └── results/')
        print(f'  └── excel/')
            
        # Place your custom callback logic here
        # e.g., load project data, etc.

load_button = widgets.Button(description='Load Project Folder', button_style='info')
load_button.on_click(on_load_clicked)

load_project_ui = widgets.VBox([
    load_folder_chooser,
    load_button,
    load_project_status
])

# Display both widgets
print("Create Project:")
display(create_project_ui)
print("\nLoad Project:")
display(load_project_ui)



Create Project:



Load Project:


In [ ]:
import os

# Widget to choose Excel file from the /excel folder of the selected project
excel_file_chooser = FileChooser()
excel_file_chooser.title = '<b>Select Excel file to parse</b>'
excel_file_chooser.filter_pattern = ['*.xlsx', '*.xls']
excel_file_chooser.show_only_dirs = False
excel_file_chooser.use_dir_icons = True
excel_file_chooser.show_hidden = False

# Status/output widgets and button
excel_status = widgets.Output()
parse_excel_button = widgets.Button(description='Parse Excel', button_style='primary')
parse_excel_status = widgets.Output()

def parse_excel_to_results():
    excel_path = excel_file_chooser.selected
    if not excel_path:
        raise RuntimeError("No Excel file selected. Please select an Excel file using the widget above.")

    project_folder = load_folder_chooser.selected_path or create_folder_chooser.selected_path
    if not project_folder:
        raise RuntimeError("No project folder selected. Please select or create a project folder.")

    results_folder = os.path.join(project_folder, "results")

    parser = OptimumSheetParser(excel_path)

    # These methods already write directly into results_folder
    # and do not create per-sheet subfolders (based on your class implementation).
    parser.save_json_per_sheet(results_folder)
    parser.save_reference_distance(results_folder)

    json_files = [f for f in os.listdir(results_folder) if f.lower().endswith(".json")]
    print(f"JSON files and reference distance saved to: {results_folder}")
    print(f"Found {len(json_files)} JSON files:")
    for f in json_files:
        print("  ", f)

def on_parse_excel_clicked(b):
    with parse_excel_status:
        clear_output()
        try:
            parse_excel_to_results()
        except Exception as e:
            print(f"Error parsing Excel: {e}")



excel_ui = widgets.VBox([
    excel_file_chooser,
    excel_status,
    parse_excel_button,
    parse_excel_status
])
parse_excel_button.on_click(on_parse_excel_clicked)
print("\nLoad Excel File:")
display(excel_ui)


Load Excel File:


In [28]:
# --- Widget 4: Generate and View STEP File (VIEW STEP via ocp_vscode) ---

import os
import pathlib
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser

import cadquery as cq
from step_cadder import carAssembly

# Output for status and viewer
step_status = widgets.Output()
step_view_status = widgets.Output()

# Button to generate STEP file
generate_step_button = widgets.Button(description="Generate STEP", button_style="primary")

def get_3d_folder():
    folder = load_folder_chooser.selected_path or create_folder_chooser.selected_path
    if folder and os.path.isdir(os.path.join(folder, "3D")):
        return os.path.join(folder, "3D")
    return None

step_folder = get_3d_folder() or os.getcwd()

step_file_chooser = FileChooser(step_folder)
step_file_chooser.title = "<b>Select STEP file to view from /3D</b>"
step_file_chooser.filter_pattern = ["*.step", "*.stp"]
step_file_chooser.show_only_dirs = False
step_file_chooser.use_dir_icons = True
step_file_chooser.show_hidden = False

def on_generate_step_clicked(b):
    with step_status:
        clear_output()
        project_folder = load_folder_chooser.selected_path or create_folder_chooser.selected_path
        if not project_folder:
            print("No project folder selected. Please select or create a project folder.")
            return

        json_dir = os.path.join(project_folder, "results")
        step_dir = os.path.join(project_folder, "3D")
        os.makedirs(step_dir, exist_ok=True)

        step_path = os.path.join(step_dir, "car_assembly.step")

        try:
            assy = carAssembly(json_dir)
            car = assy.draw(assy.setup)

            # Export STEP
            car.save(step_path)

            print(f"STEP file generated and saved to: {step_path}")
            step_file_chooser.reset(path=step_dir)

            # Auto-view generated STEP using the same viewer as the file chooser
            with step_view_status:
                clear_output()
                shape = cq.importers.importStep(step_path)
                from ocp_vscode import show_object
                show_object(shape)
        except ImportError:
            print("ocp_vscode is not installed. Please install it for 3D viewing in VS Code.")
        except Exception as e:
            print(f"Error generating or displaying STEP: {e}")

# Avoid double-binding if re-run
generate_step_button._click_handlers.callbacks = []
generate_step_button.on_click(on_generate_step_clicked)

def on_step_select(chooser):
    with step_view_status:
        clear_output()
        step_path = chooser.selected
        if not step_path:
            print("No STEP file selected.")
            return
        print(f"Selected STEP file: {step_path}")
        try:
            shape = cq.importers.importStep(step_path)
            from ocp_vscode import show_object
            show_object(shape)
        except ImportError:
            print("ocp_vscode is not installed. Please install it for 3D viewing in VS Code.")
        except Exception as e:
            print(f"Could not display STEP file: {e}")

step_file_chooser.register_callback(on_step_select)

step_ui = widgets.VBox([
    generate_step_button,
    step_status,
    step_file_chooser,
    step_view_status
])

print("\nGenerate and View STEP File:")
display(step_ui)



Generate and View STEP File:
